In [1]:
from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px
import numpy as np
import pandas as pd

In [2]:
from platform import python_version
import sys
print('Python: ' + python_version()) # Python: 3.12.
print ('plotly: ' + sys.modules["plotly"].__version__) # plotly: 5.22.0
print ('dash: ' + sys.modules["dash"].__version__) # dash: 2.17.0
print('numpy: ' + np.__version__) # numpy: 1.26.4
print('pandas: ' + pd.__version__) # pandas: 2.2.1

Python: 3.12.2
plotly: 5.22.0
dash: 2.17.0
numpy: 1.26.4
pandas: 2.2.1


## Import dataset and merge to dataframe

In [3]:
dataset_precip = pd.read_pickle("./data/UMAP_time.pkl")
datacube_precip = np.load("./data/WaterPrecip_datacube_large.npy")

In [4]:
datacube_precip = datacube_precip[datacube_precip.min(axis = (1,2)) >= 0,:,:]
dataset_precip['Precip'] = list(datacube_precip)
del datacube_precip 

## Create a geojson grid of 1° Latitude x 1° Longitude

In [5]:
geojson = {'type': 'FeatureCollection', 
                'features': []}
for x in range(180):
    for y in range(360):
        temp_geometry = {'type': 'Polygon',
                 'coordinates': [[[y-180, x-90],
                                 [y-179, x-90],
                                 [y-179, x-89],
                                 [y-180, x-89]]]}
        temp_features = {'type': 'Feature',
                 'geometry': temp_geometry,
                 'properties': {'location': "lat: " + str(x) + " lon: " + str(y)},
                 'id': x*360+y}
        geojson["features"].append(temp_features)

## Create dashboard

In [6]:
global_figure_styles = dict(margin={"r":0,"t":0,"l":0,"b":0}, paper_bgcolor= '#e4edf4')

In [7]:
# Create a Dash application instance
app = Dash(__name__)
# Define the layout of the app
app.layout = html.Div([
    # Header for the application
    html.H1(children='Data Visualisation Precipitation over time', style={'textAlign':'center'}),
    # Div for the radio button controls
    html.Div([
        dcc.RadioItems(
            id='Dimension', 
            options=["2D", "3D"],
            value="2D",
            inline=True,
            style={'display': 'inline-block', 'width': 100}
        ),
        dcc.RadioItems(
            id='Time Aggregation', 
            options=["Year", "Month"],
            value="Month",
            inline=True,
            style={'display': 'inline-block', 'border-left': '2px solid black'}
        ),
    ]),
    # Div for the graph components
    html.Div([
        dcc.Graph(id="graph_UMAP",
                  style={'display': 'inline-block', 'width': '49%'}),
        dcc.Graph(id="graph_map",
                  style={'display': 'inline-block', 'width': '49%'}),
    ])
]),


# Callback function to create the UMAP plot based on user input
@callback(
    Output("graph_UMAP", "figure"), 
    Input('Dimension', 'value'),
    Input('Time Aggregation', 'value')
)
def display_umap(dim, timeAgg):
    if dim == "3D":
        # Create a 3D scatter plot
        fig = px.scatter_3d(
            dataset_precip, x="UMAP_1", y="UMAP_2", z="UMAP_3",
            color= timeAgg, labels={'color': 'digit'},
        )
        fig.update_traces(marker_size = 3)
    else:
        # Create a 2D scatter plot
        fig = px.scatter(
            dataset_precip, x="UMAP_1", y="UMAP_2",
            color= timeAgg, labels={'color': 'digit'}
        )
    fig.update_layout(**global_figure_styles,
                      clickmode='event+select')
    return fig

# Callback function to update the choropleth map based on the selected point in the UMAP plot
@callback(
    Output("graph_map", "figure"), 
    Input('graph_UMAP', 'clickData')
)
def display_choropleth(clickData):
    # Check if a point in the UMAP plot is clicked
    if clickData is not None:
        itemp = clickData['points'][0]['pointNumber']
    else:
        itemp = 0
    # Prepare the data for the choropleth map
    temp = pd.DataFrame(dataset_precip.loc[itemp, "Precip"]).stack().reset_index()
    temp_scale = [(0, "white"), (0.5, "blue"), (1, "navy")]
    # Create a new column combining latitude and longitude information
    temp["location"] = temp.apply(lambda row: "lat: " + str(int(row["level_0"])) + " lon: " +  str(int(row["level_1"])), axis=1)
    temp.rename({0: "value"}, axis='columns', inplace=True)
    # Create a choropleth map
    fig = px.choropleth_mapbox(temp, geojson=geojson, color="value", opacity=.2,
                               color_continuous_scale=temp_scale,
                               locations="location", featureidkey="properties.location", 
                               center={"lat": 50, "lon": 10}, 
                               mapbox_style="carto-positron", zoom=3)
    fig.update_layout(**global_figure_styles)
    return fig



In [8]:
if __name__ == '__main__':
    app.run(jupyter_height=600, debug=True, port = 8070)